In [ ]:
import os
import sys
sys.path.append(os.path.abspath('drive/MyDrive/'))
sys.path.append(os.path.abspath('drive/MyDrive/cc_matrix_50.npy'))
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive',
 '/content/drive/MyDrive/cc_matrix_50.npy']

In [ ]:
from util import get_wikipedia_data,find_analogies
from brown import get_sentences_with_word2idx_limit_vocab, get_sentences_with_word2idx


# New Section

In [ ]:


import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.utils import shuffle
#from util import find_analogies

from util import get_wikipedia_data



class mymodel(tf.keras.Model):
    def __init__(self, logX,fX,reg,V,D):
        super().__init__()
        W = np.random.randn(V, D) / np.sqrt(V + D)
        b = np.zeros(V)
        U = np.random.randn(V, D) / np.sqrt(V + D)
        c = np.zeros(V)
        self.tfmu = tf.constant(logX.mean(),dtype=tf.float32)

        

        # initialize weights, inputs, 
        self.tfW = tf.Variable(W.astype(np.float32))
        self.tfb = tf.Variable(b.reshape(V, 1).astype(np.float32))
        self.tfU = tf.Variable(U.astype(np.float32))
        self.tfc = tf.Variable(c.reshape(1, V).astype(np.float32))
        self.tffX  = tf.constant(fX,dtype=tf.float32)
        self.logX = tf.constant(logX,dtype=tf.float32)
        self.reg = tf.constant(reg,dtype=tf.float32)
        

    def call(self,inputs):
        self.prediction = tf.matmul(self.tfW, tf.transpose(a=self.tfU)) + self.tfb + self.tfc + self.tfmu
        
        return  self.prediction

    def cost(self):
        try:
            costs = tf.reduce_sum(self.tffX * (self.prediction-self.logX)*(self.prediction-self.logX))
            tf.print(costs)
        except Exception:
            print("train model first")
            return


        for p in (self.tfW,self.tfU):
            costs += self.reg*tf.reduce_sum(p*p)
            tf.print(costs)
        return costs
        
    


@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(inputs)
        loss = model.cost()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss)










class Glove:
    def __init__(self, D, V, context_sz):
        self.D = D
        self.V = V
        self.context_sz = context_sz

    def fit(self, sentences, cc_matrix=None, learning_rate=1e-4, reg=0.1, xmax=100, alpha=0.75, epochs=4):
        # build co-occurrence matrix
        # paper calls it X, so we will call it X, instead of calling
        # the training data X
        # TODO: would it be better to use a sparse matrix?
        t0 = datetime.now()
        V = self.V
        D = self.D

        if not os.path.exists(cc_matrix):
            X = np.zeros((V, V))
            N = len(sentences)
            print("number of sentences to process:", N)
            it = 0
            for sentence in sentences:
                it += 1
                if it % 10000 == 0:
                    print("processed", it, "/", N)
                n = len(sentence)
                for i in range(n):
                    # i is not the word index!!!
                    # j is not the word index!!!
                    # i just points to which element of the sequence (sentence) we're looking at
                    wi = sentence[i]

                    start = max(0, i - self.context_sz)
                    end = min(n, i + self.context_sz)

                    # we can either choose only one side as context, or both
                    # here we are doing both

                    # make sure "start" and "end" tokens are part of some context
                    # otherwise their f(X) will be 0 (denominator in bias update)
                    if i - self.context_sz < 0:
                        points = 1.0 / (i + 1)
                        X[wi,0] += points
                        X[0,wi] += points
                    if i + self.context_sz > n:
                        points = 1.0 / (n - i)
                        X[wi,1] += points
                        X[1,wi] += points

                    # left side
                    for j in range(start, i):
                        wj = sentence[j]
                        points = 1.0 / (i - j) # this is +ve
                        X[wi,wj] += points
                        X[wj,wi] += points

                    # right side
                    for j in range(i + 1, end):
                        wj = sentence[j]
                        points = 1.0 / (j - i) # this is +ve
                        X[wi,wj] += points
                        X[wj,wi] += points

            # save the cc matrix because it takes forever to create
            np.save(cc_matrix, X)
        else:
            X = np.load(cc_matrix)

        print("max in X:", X.max())

        # weighting
        fX = np.zeros((V, V))
        fX[X < xmax] = (X[X < xmax] / float(xmax)) ** alpha
        fX[X >= xmax] = 1

        print("max in f(X):", fX.max())

        # target
        logX = np.log(X + 1)

        print("max in log(X):", logX.max())

        print("time to build co-occurrence matrix:", (datetime.now() - t0))

        # initialize weights
        return logX,fX,reg,self.V,self.D



def main(we_file, w2i_file, use_brown=True, n_files=50):
    if use_brown:
        cc_matrix = "cc_matrix_brown.npy"
    else:
        cc_matrix = "cc_matrix_%s.npy" % n_files

    # hacky way of checking if we need to re-load the raw data or not
    # remember, only the co-occurrence matrix is needed for training
    if os.path.exists('drive/MyDrive/cc_matrix_50.npy'):
        with open(f"drive/MyDrive/{w2i_file}") as f:
            word2idx = json.load(f)
        sentences = [] # dummy - we won't actually use it
    else:
        if use_brown:
            keep_words = set([
                'king', 'man', 'woman',
                'france', 'paris', 'london', 'rome', 'italy', 'britain', 'england',
                'french', 'english', 'japan', 'japanese', 'chinese', 'italian',
                'australia', 'australian', 'december', 'november', 'june',
                'january', 'february', 'march', 'april', 'may', 'july', 'august',
                'september', 'october',
            ])
            sentences, word2idx = get_sentences_with_word2idx_limit_vocab(n_vocab=5000, keep_words=keep_words)
        else:
            sentences, word2idx = get_wikipedia_data(n_files=n_files, n_vocab=2000)
        
        with open(w2i_file, 'w') as f:
            json.dump(word2idx, f)

    V = len(word2idx)
    model = Glove(100, V, 10)
    return model.fit(sentences, cc_matrix=cc_matrix, epochs=200)


if __name__ == '__main__':
    we = 'glove_model_50.npz'
    w2i = 'glove_word2idx_50.json'
    logX, fX,reg , V, D= main(we, w2i, use_brown=False)

    model = mymodel(logX,fX,reg,V,D) 
    optimizer = tf.keras.optimizers.Adam()
    losses = []
    for epoch in range(3):
        train_step(0)


    print(losses)

    arrays = [model.tfW.numpy(), tf.transpose(model.tfU).numpy()]
    np.savez(we, *arrays)

    # load back embeddings
    npz = np.load(we)
    W1 = npz['arr_0']
    W2 = npz['arr_1']

    with open(f"drive/MyDrive/{w2i}") as f:
        word2idx = json.load(f)
        idx2word = {i:w for w,i in word2idx.items()}

    for concat in (True, False):
        print("** concat:", concat)

        if concat:
            We = np.hstack([W1, W2.T])
        else:
            We = (W1 + W2.T) / 2


        find_analogies('king', 'man', 'woman', We, word2idx, idx2word)
        find_analogies('france', 'paris', 'london', We, word2idx, idx2word)
        find_analogies('france', 'paris', 'rome', We, word2idx, idx2word)
        find_analogies('paris', 'france', 'italy', We, word2idx, idx2word)
        find_analogies('france', 'french', 'english', We, word2idx, idx2word)
        find_analogies('japan', 'japanese', 'chinese', We, word2idx, idx2word)
        find_analogies('japan', 'japanese', 'italian', We, word2idx, idx2word)
        find_analogies('japan', 'japanese', 'australian', We, word2idx, idx2word)
        find_analogies('december', 'november', 'june', We, word2idx, idx2word)


max in X: 0.0
max in f(X): 0.0
max in log(X): 0.0
time to build co-occurrence matrix: 0:00:00.403982
0
9.53045845
19.0428352
0
8.85435772
17.6908283
0
8.21760464
16.4175377
[<tf.Tensor 'add_1:0' shape=() dtype=float32>, <tf.Tensor 'add_1:0' shape=() dtype=float32>]
** concat: True
closest match by euclidean distance: formal
king - man = formal - woman
closest match by cosine distance: formal
king - man = formal - woman
closest match by euclidean distance: kind
france - paris = kind - london
closest match by cosine distance: kind
france - paris = kind - london
closest match by euclidean distance: style
france - paris = style - rome
closest match by cosine distance: style
france - paris = style - rome
closest match by euclidean distance: officials
paris - france = officials - italy
closest match by cosine distance: officials
paris - france = officials - italy
closest match by euclidean distance: tree
france - french = tree - english
closest match by cosine distance: tree
france - french 